In [1]:
import numpy as np
import theano as theano
import theano.tensor as T
import time
import operator
from theano.gradient import grad_clip
from utils import *
from lstm_theano import *
from gru_theano import *

In [2]:
# Load data
VOCABULARY_SIZE = 2000
X_train, y_train, word_to_index, index_to_word = load_data("data/reddit-comments-2015.csv", VOCABULARY_SIZE)

Reading CSV file...
Parsed 502183 sentences.
Found 193212 unique words tokens.
Using vocabulary size 2000.
The least frequent word in our vocabulary is 'tonight' and appeared 324 times.


In [9]:
# Do a gradient check
np.random.seed(0)
model = GRUTheano(100, 10)
gradient_check_theano(model, [0,1,2,3], [1,2,3,4])

Performing gradient check for parameter E with size 1000.
Gradient check for parameter E passed.
Performing gradient check for parameter U with size 600.
Gradient check for parameter U passed.
Performing gradient check for parameter W with size 600.
Gradient check for parameter W passed.
Performing gradient check for parameter b with size 60.
Gradient check for parameter b passed.
Performing gradient check for parameter V with size 1000.
Gradient check for parameter V passed.
Performing gradient check for parameter c with size 100.
Gradient check for parameter c passed.


In [21]:
# Load parameters of pre-trained model
model = load_model_parameters_theano("GRU-2015-10-25-10-37-2000-48-128.npz")
# model = load_model_parameters_theano('./data/pretrained.npz', GRUTheano)

Building model model from GRU-2015-10-25-10-37-2000-48-128.npz with hidden_dim=128 word_dim=2000


In [22]:
print model.calculate_loss(X_train[:500], y_train[:500])

43.3986080477


In [24]:
# Load parameters of pre-trained model
# model = load_model_parameters_theano('./data/2015-10-18/GRUTheano-80-8000-10.npz', GRUTheano)
# model = load_model_parameters_theano('./data/pretrained.npz', GRUTheano)

# Build model and train

LEARNING_RATE = 1e-3
NEPOCH = 20
HIDDEN_DIM = 128

model = GRUTheano(VOCABULARY_SIZE, hidden_dim=HIDDEN_DIM, bptt_truncate=-1)

t1 = time.time()
model.sgd_step(X_train[10], y_train[10], LEARNING_RATE)
t2 = time.time()
print "SGD Step time: %f milliseconds" % ((t2 - t1) * 1000.)

train_with_sgd(model, X_train[:500], y_train[:500], LEARNING_RATE, NEPOCH, evaluate_loss_after=1, decay=0.9)

SGD Step time: 174.035072 milliseconds
2015-10-25 19:53:17: Loss after num_examples_seen=0 epoch=0: 7.596183
2015-10-25 19:54:28: Loss after num_examples_seen=500 epoch=1: 4.900535
2015-10-25 19:55:49: Loss after num_examples_seen=1000 epoch=2: 4.744880
2015-10-25 19:57:22: Loss after num_examples_seen=1500 epoch=3: 4.663191


KeyboardInterrupt: 

In [29]:
def generate_sentence(model, index_to_word, word_to_index):
    # We start the sentence with the start token
    new_sentence = [word_to_index[SENTENCE_START_TOKEN]]
    # Repeat until we get an end token
    while not new_sentence[-1] == word_to_index[SENTENCE_END_TOKEN]:
        next_word_probs = model.forward_propagation(new_sentence)[-1]
        samples = np.random.multinomial(1, next_word_probs)
        sampled_word = np.argmax(samples)
        new_sentence.append(sampled_word)
        # Seomtimes we get stuck in an infinite loop if the sentence becomes too long (e.g. .....) :(
        if len(new_sentence) > 100:
          return []
    return new_sentence

def print_sentence(s, index_to_word):
  sentence_str = [index_to_word[x] for x in s[1:-1]]
  print(" ".join(sentence_str))
  sys.stdout.flush()

def generate_sentences(model, n, index_to_word, word_to_index, min_length=5):
  for i in range(n):
      sent = []
      while len(sent) < min_length:
          sent = generate_sentence(model, index_to_word, word_to_index)
      print_sentence(sent, index_to_word)

generate_sentences(model, 10, index_to_word, word_to_index)

do . managed gt ; on was UNKNOWN_TOKEN that n't more park it you so too from so , and attempt ) , 's who you ... friends has it from type .
UNKNOWN_TOKEN keep right , guy that , & has she UNKNOWN_TOKEN to final .
i holding that request and any 's be .. the ready the i degree their make 25 his UNKNOWN_TOKEN the at for eat a over of UNKNOWN_TOKEN it .
she world a too account , i please : from still UNKNOWN_TOKEN lost `` various .
this building 's the '' of UNKNOWN_TOKEN .
gt just them is as the ^ done UNKNOWN_TOKEN .
if i & be can UNKNOWN_TOKEN upset show and been be and and UNKNOWN_TOKEN but you terms UNKNOWN_TOKEN to entire be show .
've were along religion , it teach be these .
UNKNOWN_TOKEN and they a trade career .
ways and have UNKNOWN_TOKEN would ) is my a really has year and i under
